# 1

In [53]:
%store -r df_2017

In [ ]:
#1 
from datetime import date, timedelta

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import lightgbm as lgb


def to_bool(x):
    if "True" in x:
        return True
    else:
        return False


input_path = "/Users/dongjian/data/sales_forcasting/"
df_train = pd.read_csv(
    input_path + 'train.csv',
    usecols=[1, 2, 3, 4, 5],
    dtype={
        'id': 'int64',
        'item_nbr': 'int32',
        'store_nbr': 'int8',
        'unit_sales': 'float32',
        'onpromotion': bool,
    },
    converters={
        'unit_sales': lambda u: np.log1p(float(u)) if float(u) > 0 else 0,
        "onpromotion": to_bool
    },
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)

stores = pd.read_csv(
    input_path + 'stores.csv',
    dtype={
        'cluster': 'int32',
        'store_nbr': 'int8',
    })
transactions = pd.read_csv(
    input_path + 'transactions.csv',
    parse_dates=['date'],
    dtype={'transactions': 'int16',
           'store_nbr': 'int8'},
    infer_datetime_format=True)

items = pd.read_csv(
    input_path + "items.csv",
    dtype={
        'item_nbr': 'int32',
        'perishable': 'int8',
        'class': 'int16'
    })

df_test = pd.read_csv(
    input_path + "test.csv",
    usecols=[0, 1, 2, 3, 4],
    dtype={
        'onpromotion': bool
    },
    parse_dates=["date"]  # , date_parser=parser
)
# .set_index(['store_nbr', 'item_nbr', 'date'])

In [9]:
df_test = pd.read_csv(
    input_path + "test.csv",
    usecols=[0, 1, 2, 3, 4],
    dtype={
        'onpromotion': bool
    },
    parse_dates=["date"]  # , date_parser=parser
)

In [17]:
df_2017 = df_train_new[df_train.date.isin(
    pd.date_range("2017-05-31", periods=7 * 11))].copy()

In [24]:
df_2017.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion,city,state,type,cluster,transactions,family,class,perishable
50912462,2017-05-31,1,96995,0.693147,False,Quito,Pichincha,D,13,1878.0,GROCERY I,1093,0
50912463,2017-05-31,1,99197,0.693147,False,Quito,Pichincha,D,13,1878.0,GROCERY I,1067,0
50912464,2017-05-31,1,103520,1.386294,False,Quito,Pichincha,D,13,1878.0,GROCERY I,1028,0
50912465,2017-05-31,1,103665,2.197225,False,Quito,Pichincha,D,13,1878.0,BREAD/BAKERY,2712,1
50912466,2017-05-31,1,105574,1.386294,False,Quito,Pichincha,D,13,1878.0,GROCERY I,1045,0


In [51]:
df_2017[(df_2017.store_nbr==1)&(df_2017.date.isin([date(2017, 5, 31)]))].head()

,date,store_nbr,item_nbr,unit_sales,onpromotion,city,state,type,cluster,transactions,family,class,perishable
50912462,2017-05-31,1,96995,0.693147,False,Quito,Pichincha,D,13,1878.0,GROCERY I,1093,0
50912463,2017-05-31,1,99197,0.693147,False,Quito,Pichincha,D,13,1878.0,GROCERY I,1067,0
50912464,2017-05-31,1,103520,1.386294,False,Quito,Pichincha,D,13,1878.0,GROCERY I,1028,0
50912465,2017-05-31,1,103665,2.197225,False,Quito,Pichincha,D,13,1878.0,BREAD/BAKERY,2712,1
50912466,2017-05-31,1,105574,1.386294,False,Quito,Pichincha,D,13,1878.0,GROCERY I,1045,0


In [43]:
tmp_promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)

In [36]:
df_2017.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion,city,state,type,cluster,transactions,family,class,perishable
50912462,2017-05-31,1,96995,0.693147,False,Quito,Pichincha,D,13,1878.0,GROCERY I,1093,0
50912463,2017-05-31,1,99197,0.693147,False,Quito,Pichincha,D,13,1878.0,GROCERY I,1067,0
50912464,2017-05-31,1,103520,1.386294,False,Quito,Pichincha,D,13,1878.0,GROCERY I,1028,0
50912465,2017-05-31,1,103665,2.197225,False,Quito,Pichincha,D,13,1878.0,BREAD/BAKERY,2712,1
50912466,2017-05-31,1,105574,1.386294,False,Quito,Pichincha,D,13,1878.0,GROCERY I,1045,0


In [62]:
df.head()

date 
 2017-05-31 00:00:00 
 2017-06-01 00:00:00 
 2017-06-02 00:00:00 
 2017-06-03 00:00:00 
 2017-06-04 00:00:00 
 2017-06-05 00:00:00 
 2017-06-06 00:00:00 
 2017-06-07 00:00:00 
 2017-06-08 00:00:00 
 2017-06-09 00:00:00 
 ... 
 2017-08-06 00:00:00 
 2017-08-07 00:00:00 
 2017-08-08 00:00:00 
 2017-08-09 00:00:00 
 2017-08-10 00:00:00 
 2017-08-11 00:00:00 
 2017-08-12 00:00:00 
 2017-08-13 00:00:00 
 2017-08-14 00:00:00 
 2017-08-15 00:00:00 
 
 
 store_nbr 
 item_nbr 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 96995 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 ... 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 
 
 99197 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 ... 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 
 
 103520 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 ... 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 
 
 103665 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 ... 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 
 
 105574 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 ... 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 
 
 
 5 rows × 77 columns

In [3]:
promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)

In [83]:
promo_2017_train.head()

date 
 2017-05-31 00:00:00 
 2017-06-01 00:00:00 
 2017-06-02 00:00:00 
 2017-06-03 00:00:00 
 2017-06-04 00:00:00 
 2017-06-05 00:00:00 
 2017-06-06 00:00:00 
 2017-06-07 00:00:00 
 2017-06-08 00:00:00 
 2017-06-09 00:00:00 
 ... 
 2017-08-06 00:00:00 
 2017-08-07 00:00:00 
 2017-08-08 00:00:00 
 2017-08-09 00:00:00 
 2017-08-10 00:00:00 
 2017-08-11 00:00:00 
 2017-08-12 00:00:00 
 2017-08-13 00:00:00 
 2017-08-14 00:00:00 
 2017-08-15 00:00:00 
 
 
 store_nbr 
 item_nbr 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 96995 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 ... 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 
 
 99197 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 ... 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 
 
 103520 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 ... 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 
 
 103665 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 ... 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 
 
 105574 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 ... 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 False 
 
 
 
 5 rows × 77 columns

In [36]:
df_2017 = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)

items = items.reindex(df_2017.index.get_level_values(1))

In [61]:
df_2017[
            pd.date_range(date(2017, 6, 21), periods=16)
        ].values

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.69314718,  0.        ,  0.        , ...,  0.        ,
         0.        ,  1.38629436],
       [ 0.69314718,  1.38629436,  1.38629436, ...,  0.        ,
         0.69314718,  1.94591015],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  2.30258509,
         0.69314718,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [41]:
def get_timespan(df, dt, minus, periods):
    return df[
        pd.date_range(dt - timedelta(days=minus), periods=periods)
    ]

def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        "mean_3_2017": get_timespan(df_2017, t2017, 3, 3).mean(axis=1).values,
        "mean_7_2017": get_timespan(df_2017, t2017, 7, 7).mean(axis=1).values,
        "mean_14_2017": get_timespan(df_2017, t2017, 14, 14).mean(axis=1).values,
        "promo_14_2017": get_timespan(promo_2017, t2017, 14, 14).sum(axis=1).values
    })
    for i in range(16):
        X["promo_{}".format(i)] = promo_2017[
            t2017 + timedelta(days=i)].values.astype(np.uint8)
    if is_train: # 取后续的16天来进行评价。
        y = df_2017[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    return X

print("Preparing dataset...")
t2017 = date(2017, 6, 21)
X_l, y_l = [], []
for i in range(4):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(
        t2017 + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)#4分训练数据，没份都对应了后续16天的结果。
y_train = np.concatenate(y_l, axis=0)
# del X_l, y_l
X_val, y_val = prepare_dataset(date(2017, 7, 26))
X_test = prepare_dataset(date(2017, 8, 16), is_train=False)

Preparing dataset...


In [5]:
print("Training and predicting models...")
params = {
    'num_leaves': 2**7 - 1,
    'objective': 'regression_l2',
#     'max_depth': 8,
    'min_data_in_leaf': 50,
    'learning_rate': 0.05,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.75,
    'bagging_freq': 1,
    'metric': 'l2',
    'num_threads': 4
}

MAX_ROUNDS = 1000
val_pred = []
test_pred = []
cate_vars = []
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * 4) * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        weight=items["perishable"] * 0.25 + 1,
        categorical_feature=cate_vars)
    bst = lgb.train(
        params, dtrain, num_bo=ost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval], early_stopping_rounds=50, verbose_eval=50
    )
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

print("Validation mse:", mean_squared_error(
    y_val, np.array(val_pred).transpose()))

print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('lgb.csv', float_format='%.4f', index=None)

Preparing dataset...
Training and predicting models...
Step 1


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/lightgbm/basic.py:1021: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds.
[50]	training's l2: 0.345189	valid_1's l2: 0.341428
[100]	training's l2: 0.333333	valid_1's l2: 0.330096
[150]	training's l2: 0.331382	valid_1's l2: 0.328751
[200]	training's l2: 0.330357	valid_1's l2: 0.328286
[250]	training's l2: 0.329506	valid_1's l2: 0.327894
[300]	training's l2: 0.328802	valid_1's l2: 0.327651
[350]	training's l2: 0.328191	valid_1's l2: 0.327479
[400]	training's l2: 0.327642	valid_1's l2: 0.327311
[450]	training's l2: 0.327144	valid_1's l2: 0.327257
[500]	training's l2: 0.326652	valid_1's l2: 0.327189
[550]	training's l2: 0.326221	valid_1's l2: 0.32715
[600]	training's l2: 0.325823	valid_1's l2: 0.327099
[650]	training's l2: 0.325429	valid_1's l2: 0.327055
[700]	training's l2: 0.325027	valid_1's l2: 0.327026
[750]	training's l2: 0.32468	valid_1's l2: 0.327036
Early stopping, best iteration is:
[742]	training's l2: 0.324728	valid_1's l2: 0.327018
mean_14_2017: 1988605.41
mean_7_2017: 1456113.56
mean_3_20

[50]	training's l2: 0.399236	valid_1's l2: 0.499917
[100]	training's l2: 0.387595	valid_1's l2: 0.484574
[150]	training's l2: 0.384612	valid_1's l2: 0.483293
[200]	training's l2: 0.382999	valid_1's l2: 0.483023
[250]	training's l2: 0.381817	valid_1's l2: 0.482707
[300]	training's l2: 0.380931	valid_1's l2: 0.482696
Early stopping, best iteration is:
[272]	training's l2: 0.381385	valid_1's l2: 0.482499
mean_14_2017: 2180053.39
mean_7_2017: 783648.91
mean_3_2017: 311571.39
promo_6: 154218.90
promo_14_2017: 50307.58
promo_3: 14124.95
promo_7: 10381.33
promo_13: 9031.56
promo_5: 7778.12
promo_0: 5256.53
promo_4: 4478.50
promo_1: 4349.53
promo_9: 3761.66
promo_2: 3748.48
promo_14: 3263.83
promo_8: 2346.24
promo_11: 1333.58
promo_15: 1228.89
promo_12: 1161.88
promo_10: 1110.30
Step 8
Training until validation scores don't improve for 50 rounds.
[50]	training's l2: 0.388046	valid_1's l2: 0.460928
[100]	training's l2: 0.375757	valid_1's l2: 0.442314
[150]	training's l2: 0.37291	valid_1's l2: 0

Step 15
Training until validation scores don't improve for 50 rounds.
[50]	training's l2: 0.410926	valid_1's l2: 0.410558
[100]	training's l2: 0.397122	valid_1's l2: 0.398618
[150]	training's l2: 0.394345	valid_1's l2: 0.397655
[200]	training's l2: 0.392782	valid_1's l2: 0.397482
[250]	training's l2: 0.39166	valid_1's l2: 0.397365
[300]	training's l2: 0.390699	valid_1's l2: 0.397227
[350]	training's l2: 0.389916	valid_1's l2: 0.396985
[400]	training's l2: 0.389194	valid_1's l2: 0.396816
[450]	training's l2: 0.388537	valid_1's l2: 0.39676
[500]	training's l2: 0.387919	valid_1's l2: 0.396731
[550]	training's l2: 0.387354	valid_1's l2: 0.396791
Early stopping, best iteration is:
[522]	training's l2: 0.38765	valid_1's l2: 0.396695
mean_14_2017: 2327391.81
mean_7_2017: 699228.52
mean_3_2017: 244621.72
promo_14: 232602.41
promo_14_2017: 50764.80
promo_7: 31640.63
promo_0: 30333.36
promo_15: 20345.57
promo_13: 11162.36
promo_9: 9715.37
promo_12: 8515.95
promo_10: 6340.64
promo_2: 4376.13
prom

In [5]:
t = lambda x,y:(x-y)/y
t(0.04653213585500124	,0.046430892362417486)

0.0021805200682660547